### 系统路径以及魔法重载

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
import os
# 获取当前notebook文件的目录
current_dir = os.path.dirname(os.path.abspath('.'))
sys.path.append(current_dir)
os.chdir(current_dir)

### 数据加载与去重

In [ ]:
all_data_url = './data/ignore/20250926_AIO_correct_ex0928_85.json'
import json
with open(all_data_url, 'r', encoding='utf-8') as f:
    all_data = json.load(f)
len(all_data)

In [ ]:
# from utils import deduplicate_dicts
# data = deduplicate_dicts(all_data,'query')
# len(data)


### 擦除三步骤

In [ ]:
from template_construct.query_erase import query_erase
data = query_erase(all_data)

In [ ]:
data[0]

### 词嵌入并保存

In [ ]:
from template_construct.embed_tool import get_embeddings
raw_embeddings = get_embeddings([item['query'] for item in data])

erased_embeddings = get_embeddings([item['erased'] for item in data])

In [ ]:
for i in range(len(data)):
    data[i]['query_embedding'] = raw_embeddings[i]
    data[i]['erased_embedding'] = erased_embeddings[i]

In [ ]:
target_data_file = './data/ignore/20250916_AIO_correct_erased_embedding.json'
with open(target_data_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:
import pandas as pd
data = pd.DataFrame(data)

In [ ]:
if 'rewritten' in data.columns:
    data['rewritten'] = data['rewritten'].astype(str)
if 'config' in data.columns:
    data['config'] = data['config'].astype(str)

In [ ]:
data.to_parquet(
    './data/ignore/20250926_AIO_correct_erased_embedding_ex0928_85.parquet', engine='pyarrow')

### 加载数据并处理

In [ ]:
import json
import pandas as pd
target_data_file = './data/ignore/20250916_AIO_correct_erased_embedding.json'
data = pd.read_json(target_data_file)

In [ ]:
data.to_parquet('./data/ignore/20250916_AIO_correct_erased_embedding.parquet')

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

# 转换为Parquet
def json_to_parquet(json_file, parquet_file):
    # 分块读取大JSON文件
    chunks = pd.read_json(json_file)
    if 'rewritten' in chunks.columns:
        chunks['rewritten'] = chunks['rewritten'].astype(str)
    if 'config' in chunks.columns:
        chunks['config'] = chunks['config'].astype(str)
    chunks.to_parquet(f"{parquet_file}", engine='pyarrow')
# 读取特定列（减少内存占用）

def read_parquet_columns(parquet_file, columns):
    return pd.read_parquet(parquet_file, columns=columns)

In [ ]:
target_json_file = '../data/ignore/20250916_AIO_correct_erased_embedding.json'
target_parquet_file = '../data/ignore/20250916_AIO_correct_erased_embedding.parquet'
json_to_parquet(target_json_file, target_parquet_file)

In [ ]:
sample_json_file = './data/20250909_AIMI全量_AIO_erased_embedding_sample.json'
data = pd.read_json(target_json_file)
sample_data = data.sample(n=100, random_state=42)
sample_data.to_json(sample_json_file, orient='records')


In [ ]:
import pandas as pd

data = pd.read_parquet(target_parquet_file)
data.columns


### 使用训练好的模型

In [ ]:
import pandas as pd
target_parquet_file = './data/ignore/20250909_AIMI全量_AIO_erased_embedding.parquet'
target_df = pd.read_parquet(target_parquet_file)
erased_embeddings = target_df['erased_embedding'].tolist()
len(erased_embeddings[0])